In [1]:
!unzip -o *.zip


Archive:  archive (1).zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [2]:
!ls


'archive (1).zip'		    olist_orders_dataset.csv
 olist_customers_dataset.csv	    olist_products_dataset.csv
 olist_geolocation_dataset.csv	    olist_sellers_dataset.csv
 olist_order_items_dataset.csv	    product_category_name_translation.csv
 olist_order_payments_dataset.csv   sample_data
 olist_order_reviews_dataset.csv


In [3]:
!pip -q install duckdb


In [4]:
import duckdb

con = duckdb.connect("olist_dw.duckdb")

con.execute("""
CREATE OR REPLACE TABLE raw_customers AS
SELECT * FROM read_csv_auto('olist_customers_dataset.csv');
""")

con.execute("SELECT COUNT(*) AS customers_rows FROM raw_customers;").fetchall()


[(99441,)]

In [5]:
con.execute("""
CREATE OR REPLACE TABLE raw_orders AS
SELECT * FROM read_csv_auto('olist_orders_dataset.csv');
""")

con.execute("SELECT COUNT(*) FROM raw_orders;").fetchall()


[(99441,)]

In [6]:
con.execute("""
CREATE OR REPLACE TABLE raw_order_items AS
SELECT * FROM read_csv_auto('olist_order_items_dataset.csv');
""")

con.execute("SELECT COUNT(*) FROM raw_order_items;").fetchall()


[(112650,)]

In [8]:
con.execute("""
CREATE OR REPLACE TABLE raw_payments AS
SELECT * FROM read_csv_auto('olist_order_payments_dataset.csv');
""")


In [9]:
con.execute("SELECT COUNT(*) FROM raw_payments;").fetchall()


[(103886,)]

In [10]:
con.execute("""
CREATE OR REPLACE TABLE raw_products AS
SELECT * FROM read_csv_auto('olist_products_dataset.csv');
""")


In [11]:
con.execute("SELECT COUNT(*) FROM raw_products;").fetchall()


[(32951,)]

In [12]:
con.execute("""
CREATE OR REPLACE TABLE raw_category_translation AS
SELECT * FROM read_csv_auto('product_category_name_translation.csv');
""")


In [13]:
con.execute("SELECT COUNT(*) FROM raw_category_translation;").fetchall()


[(71,)]

In [14]:
con.execute("SELECT COUNT(*) FROM raw_products;").fetchall()


[(32951,)]

In [15]:
con.execute("SHOW TABLES;").fetchall()


[('raw_category_translation',),
 ('raw_customers',),
 ('raw_order_items',),
 ('raw_orders',),
 ('raw_payments',),
 ('raw_products',)]

In [16]:
con.execute("SELECT * FROM raw_products LIMIT 5;").fetchdf()


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [17]:
con.execute("""
CREATE OR REPLACE TABLE stg_products AS
SELECT
  p.product_id,
  COALESCE(t.product_category_name_english, p.product_category_name) AS product_category,
  p.product_name_lenght,
  p.product_description_lenght,
  p.product_photos_qty
FROM raw_products p
LEFT JOIN raw_category_translation t
  ON p.product_category_name = t.product_category_name;
""")


In [18]:
con.execute("SELECT COUNT(*) FROM stg_products;").fetchall()


[(32951,)]

In [19]:
con.execute("SELECT * FROM stg_products LIMIT 5;").fetchdf()


,product_id,product_category,product_name_lenght,product_description_lenght,product_photos_qty
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40,287,1
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44,276,1
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46,250,1
3,cef67bcfe19066a932b7673e239eb23d,baby,27,261,1
4,9dc1a7de274444849c219cff195d0b71,housewares,37,402,4


In [20]:
con.execute("""
CREATE OR REPLACE TABLE stg_orders AS
SELECT
  order_id,
  customer_id,
  order_status,
  order_purchase_timestamp
FROM raw_orders;
""")


In [21]:
con.execute("SELECT COUNT(*) FROM stg_orders;").fetchall()


[(99441,)]

In [22]:
con.execute("SELECT * FROM stg_orders LIMIT 5;").fetchdf()


,order_id,customer_id,order_status,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39


In [23]:
con.execute("""
CREATE OR REPLACE TABLE stg_order_items AS
SELECT
  order_id,
  order_item_id,
  product_id,
  seller_id,
  shipping_limit_date,
  price,
  freight_value
FROM raw_order_items;
""")


In [24]:
con.execute("""
CREATE OR REPLACE TABLE fact_order_items AS
SELECT
  oi.order_id,
  oi.order_item_id,
  o.customer_id,
  o.order_status,
  o.order_purchase_timestamp,
  oi.product_id,
  oi.seller_id,
  oi.price,
  oi.freight_value,
  (oi.price + oi.freight_value) AS item_total
FROM stg_order_items oi
JOIN stg_orders o
  ON oi.order_id = o.order_id;
""")


In [25]:
con.execute("SELECT COUNT(*) FROM fact_order_items;").fetchall()


[(112650,)]

Create customer dimension

In [26]:
con.execute("""
CREATE OR REPLACE TABLE dim_customers AS
SELECT
  customer_id,
  customer_unique_id,
  customer_city,
  customer_state
FROM raw_customers;
""")


Create product dimension (from your cleaned table)

In [27]:
con.execute("""
CREATE OR REPLACE TABLE dim_products AS
SELECT
  product_id,
  product_category
FROM stg_products;
""")


Quick join check (fact + dims)

In [28]:
con.execute("""
SELECT
  COUNT(*) AS fact_rows,
  COUNT(DISTINCT customer_id) AS customers_in_fact,
  COUNT(DISTINCT product_id) AS products_in_fact
FROM fact_order_items;
""").fetchdf()


,fact_rows,customers_in_fact,products_in_fact
0,112650,98666,32951


Create a monthly revenue KPI table

In [29]:
con.execute("""
CREATE OR REPLACE TABLE mart_monthly_revenue AS
SELECT
  date_trunc('month', order_purchase_timestamp) AS month,
  SUM(item_total) AS revenue,
  COUNT(DISTINCT order_id) AS orders,
  SUM(item_total) / NULLIF(COUNT(DISTINCT order_id), 0) AS aov
FROM fact_order_items
WHERE order_status IN ('delivered','shipped','invoiced','approved')
GROUP BY 1
ORDER BY 1;
""")


Show the top 12 months

In [30]:
con.execute("""
SELECT * FROM mart_monthly_revenue
ORDER BY month
LIMIT 12;
""").fetchdf()


,month,revenue,orders,aov
0,2016-09-01,279.69,2,139.845000
1,2016-10-01,50941.90,288,176.881597
2,2016-12-01,19.62,1,19.620000
3,2017-01-01,134428.37,778,172.787108
4,2017-02-01,276890.29,1686,164.229116
5,2017-03-01,419742.01,2594,161.812648
6,2017-04-01,399682.66,2367,168.856215
7,2017-05-01,578440.07,3617,159.922607
8,2017-06-01,496987.09,3193,155.648948
9,2017-07-01,576991.44,3935,146.630607


Add revenue growth % (Window function)

In [31]:
con.execute("""
SELECT
  month,
  revenue,
  ROUND(
    100 * (revenue - LAG(revenue) OVER (ORDER BY month))
    / NULLIF(LAG(revenue) OVER (ORDER BY month), 0),
  2) AS mom_growth_pct
FROM mart_monthly_revenue
ORDER BY month;
""").fetchdf()


,month,revenue,mom_growth_pct
0,2016-09-01,279.69,NaN
1,2016-10-01,50941.90,18113.70
2,2016-12-01,19.62,-99.96
3,2017-01-01,134428.37,685059.89
4,2017-02-01,276890.29,105.98
5,2017-03-01,419742.01,51.59
6,2017-04-01,399682.66,-4.78
7,2017-05-01,578440.07,44.72
8,2017-06-01,496987.09,-14.08
9,2017-07-01,576991.44,16.10


Revenue by product category (Top 15)

In [32]:
con.execute("""
SELECT
  p.product_category,
  SUM(f.item_total) AS revenue,
  COUNT(DISTINCT f.order_id) AS orders
FROM fact_order_items f
JOIN dim_products p ON f.product_id = p.product_id
WHERE f.order_status IN ('delivered','shipped','invoiced','approved')
GROUP BY 1
ORDER BY revenue DESC
LIMIT 15;
""").fetchdf()


,product_category,revenue,orders
0,health_beauty,1435578.68,8786
1,watches_gifts,1292261.61,5574
2,bed_bath_table,1239673.62,9392
3,sports_leisure,1142467.59,7653
4,computers_accessories,1046764.49,6637
5,furniture_decor,891957.76,6397
6,housewares,769808.85,5830
7,cool_stuff,701642.50,3609
8,auto,676626.81,3858
9,garden_tools,576355.04,3493


Revenue by customer state (Top 10)

In [33]:
con.execute("""
SELECT
  c.customer_state,
  SUM(f.item_total) AS revenue,
  COUNT(DISTINCT f.order_id) AS orders
FROM fact_order_items f
JOIN dim_customers c ON f.customer_id = c.customer_id
WHERE f.order_status IN ('delivered','shipped','invoiced','approved')
GROUP BY 1
ORDER BY revenue DESC
LIMIT 10;
""").fetchdf()


,customer_state,revenue,orders
0,SP,5846695.32,40989
1,RJ,2107423.23,12665
2,MG,1837093.01,11465
3,RS,875014.48,5401
4,PR,791053.15,4967
5,BA,604291.24,3333
6,SC,603424.11,3585
7,DF,350594.07,2113
8,GO,339033.35,1993
9,ES,322962.52,2016


AOV by state (Top 10)

In [35]:
con.execute("""
SELECT
  c.customer_state,
  SUM(f.item_total) / NULLIF(COUNT(DISTINCT f.order_id),0) AS aov
FROM fact_order_items f
JOIN dim_customers c ON f.customer_id = c.customer_id
WHERE f.order_status IN ('delivered','shipped','invoiced','approved')
GROUP BY 1
ORDER BY aov DESC
LIMIT 10;
""").fetchdf()


,customer_state,aov
0,PB,264.771456
1,AC,242.835802
2,AP,240.922537
3,AL,234.779657
4,RO,234.083443
5,PA,224.296763
6,RR,221.088222
7,TO,220.241408
8,PI,219.563326
9,SE,211.687884


In [36]:
con.execute("""
SELECT
  order_id,
  order_item_id,
  COUNT(*) AS cnt
FROM fact_order_items
GROUP BY order_id, order_item_id
HAVING COUNT(*) > 1;
""").fetchdf()


,order_id,order_item_id,cnt


In [37]:
con.execute("""
SELECT
  SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS null_orders,
  SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS null_customers,
  SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS null_products
FROM fact_order_items;
""")


In [38]:
con.execute("""
SELECT
  MIN(item_total) AS min_revenue,
  MAX(item_total) AS max_revenue
FROM fact_order_items;
""").fetchdf()


,min_revenue,max_revenue
0,6.08,6929.31


In [40]:
con.execute("DROP TABLE mart_monthly_revenue;")


In [41]:
con.execute("""
CREATE VIEW mart_monthly_revenue AS
SELECT
  DATE_TRUNC('month', order_purchase_timestamp) AS month,
  SUM(item_total) AS total_revenue,
  COUNT(DISTINCT order_id) AS total_orders
FROM fact_order_items
GROUP BY 1
ORDER BY 1;
""")


In [42]:
con.execute("""
SELECT * FROM mart_monthly_revenue
LIMIT 5;
""").fetchdf()


,month,total_revenue,total_orders
0,2016-09-01,354.75,3
1,2016-10-01,56808.84,308
2,2016-12-01,19.62,1
3,2017-01-01,137188.49,789
4,2017-02-01,286280.62,1733


In [43]:
con.execute("SELECT * FROM mart_monthly_revenue LIMIT 12;").fetchdf()


,month,total_revenue,total_orders
0,2016-09-01,354.75,3
1,2016-10-01,56808.84,308
2,2016-12-01,19.62,1
3,2017-01-01,137188.49,789
4,2017-02-01,286280.62,1733
5,2017-03-01,432048.59,2641
6,2017-04-01,412422.24,2391
7,2017-05-01,586190.95,3660
8,2017-06-01,502963.04,3217
9,2017-07-01,584971.62,3969


In [44]:
con.execute("""
SELECT p.product_category, SUM(f.item_total) AS revenue
FROM fact_order_items f
JOIN dim_products p ON f.product_id = p.product_id
GROUP BY 1
ORDER BY revenue DESC
LIMIT 10;
""").fetchdf()


,product_category,revenue
0,health_beauty,1441248.07
1,watches_gifts,1305541.61
2,bed_bath_table,1241681.72
3,sports_leisure,1156656.48
4,computers_accessories,1059272.40
5,furniture_decor,902511.79
6,housewares,778397.77
7,cool_stuff,719329.95
8,auto,685384.32
9,garden_tools,584219.21


In [45]:
con.execute("""
SELECT * FROM fact_order_items LIMIT 5;
""").fetchdf()


,order_id,order_item_id,customer_id,order_status,order_purchase_timestamp,product_id,seller_id,price,freight_value,item_total
0,00010242fe8c5a6d1ba2dd792cb16214,1,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,218.04


In [46]:
con.execute("""
CREATE OR REPLACE VIEW mart_kpi_summary AS
SELECT
  COUNT(DISTINCT order_id) AS total_orders,
  COUNT(DISTINCT customer_id) AS total_customers,
  SUM(item_total) AS total_revenue,
  ROUND(SUM(item_total) / COUNT(DISTINCT order_id), 2) AS avg_order_value
FROM fact_order_items;
""")


In [47]:
con.execute("""
SELECT * FROM mart_kpi_summary;
""").fetchdf()


,total_orders,total_customers,total_revenue,avg_order_value
0,98666,98666,1.584355e+07,160.58


In [48]:
con.execute("""
CREATE OR REPLACE VIEW mart_state_revenue AS
SELECT
  c.customer_state,
  SUM(f.item_total) AS revenue
FROM fact_order_items f
JOIN dim_customers c ON f.customer_id = c.customer_id
GROUP BY c.customer_state
ORDER BY revenue DESC;
""")
